# 🚀 Fine-Tuning LLM con Unsloth + LoRA

**Modelo**: unsloth/Llama-3.2-3B-Instruct (4-bit quantized)
**Dataset**: ledgar_finetune_train.json (~60K ejemplos del LEDGAR dataset)
**Método**: QLoRA (4-bit) con Unsloth para 2x velocidad

### Instrucciones:
1. Sube `sentiment_500.json` como un Kaggle Dataset
2. Añade ese dataset a este notebook (botón '+Add Input')
3. Selecciona GPU T4 x2 como acelerador
4. Activa Internet
5. Ejecuta todas las celdas

In [2]:
import torch
assert torch.cuda.is_available(), "❌ GPU NO detectada. Ve a Settings → Accelerator → GPU T4 x2"
torch.cuda.set_device(0)
print(f"✅ GPU activa: {torch.cuda.get_device_name(0)}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

✅ GPU activa: Tesla T4


AttributeError: 'torch._C._CudaDeviceProperties' object has no attribute 'total_mem'

## Cell 1: Setup & Instalación

In [3]:
%%capture
!pip install --upgrade pip
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade "transformers>=4.46" "trl>=0.12.0" peft accelerate bitsandbytes
!pip install minio
print('✅ Instalación completada (Versiones Modernas)')

## Cell 2: Imports & Config

In [9]:
import torch
import json
import os
import time
from datetime import datetime
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

# ═══════════════════════════════════════════
# 📝 CONFIGURACIÓN - Edita estos valores
# ═══════════════════════════════════════════
MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"  # Modelo preoptimizado 4-bit
MAX_SEQ_LENGTH = 1024
EPOCHS = 1
BATCH_SIZE = 4
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-4
LORA_R = 16
LORA_ALPHA = 16

# Ruta al dataset (ajusta según cómo lo subas)
# Si lo subes como Kaggle Dataset, la ruta será algo como:
# /kaggle/input/{tu-usuario}-{nombre-dataset}/{archivo}.json
DATASET_PATH = "/kaggle/input/datasets/maradg/ledgar/ledgar_finetune_train.json"
VAL_PATH = "/kaggle/input/datasets/maradg/ledgar/ledgar_finetune_val.json"
TEST_PATH = "/kaggle/input/datasets/maradg/ledgar/ledgar_finetune_test.json"

# Auto-find
for root, dirs, files in os.walk("/kaggle/input/"):
    for f in files:
        if f == "ledgar_finetune_train.json":
            DATASET_PATH = os.path.join(root, f)
        elif f == "ledgar_finetune_val.json":
            VAL_PATH = os.path.join(root, f)
        elif f == "ledgar_finetune_test.json":
            TEST_PATH = os.path.join(root, f)

print(f"📂 Dataset encontrado: {DATASET_PATH}")
print(f"🧠 Modelo: {MODEL_NAME}")
print(f"🔧 LoRA r={LORA_R}, alpha={LORA_ALPHA}")
print(f"📊 Epochs: {EPOCHS}, Batch: {BATCH_SIZE}, LR: {LEARNING_RATE}")
print(f"🖥️ GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB" if torch.cuda.is_available() else '')

📂 Dataset encontrado: /kaggle/input/datasets/maradg/ledgar/ledgar_finetune_train.json
🧠 Modelo: unsloth/Llama-3.2-3B-Instruct-bnb-4bit
🔧 LoRA r=16, alpha=16
📊 Epochs: 1, Batch: 4, LR: 0.0002
🖥️ GPU: Tesla T4
💾 VRAM: 15.6 GB


## Cell 3: Cargar Modelo (4-bit Quantized)

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect: float16 para T4, bfloat16 para Ampere+
    load_in_4bit=True,
)
print(f"✅ Modelo cargado: {MODEL_NAME}")

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 5.2.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Will load unsloth/llama-3.2-3b-instruct-bnb-4bit as a legacy tokenizer.


✅ Modelo cargado: unsloth/Llama-3.2-3B-Instruct-bnb-4bit


## Cell 4: Aplicar LoRA Adapters

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=LORA_ALPHA,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Contar parámetros entrenables
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA aplicado: {trainable:,} params entrenables / {total:,} total ({100*trainable/total:.2f}%)")

Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


✅ LoRA aplicado: 24,313,856 params entrenables / 1,827,777,536 total (1.33%)


## Cell 5: Preparar Dataset

In [13]:
with open(DATASET_PATH, 'r') as f:
    raw_data = json.load(f)
with open(VAL_PATH, 'r') as f:
    val_data = json.load(f)

print(f"📊 Train: {len(raw_data)} ejemplos")
print(f"📊 Val: {len(val_data)} ejemplos")

def format_messages(data):
    formatted = []
    for item in data:
        msgs = item["messages"]
        user_msg = ""
        assistant_msg = ""
        for m in msgs:
            if m["role"] == "user":
                user_msg = m["content"]
            elif m["role"] == "assistant":
                assistant_msg = m["content"]
        text = f"### Instruction:\n{user_msg}\n\n### Response:\n{assistant_msg}"
        formatted.append({"text": text})
    return formatted

dataset = Dataset.from_list(format_messages(raw_data))
val_dataset = Dataset.from_list(format_messages(val_data))

print(f"✅ Train formateado: {len(dataset)} | Val formateado: {len(val_dataset)}")

📊 Train: 60000 ejemplos
📊 Val: 10000 ejemplos
✅ Train formateado: 60000 | Val formateado: 10000


## Cell 6: Configurar Trainer

In [14]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

# Importante: SFTConfig reemplaza a parte de los argumentos antiguos
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=val_dataset,  # ← Ya podemos usar esto
    args=SFTConfig(
        dataset_text_field="text",
        max_seq_length=MAX_SEQ_LENGTH,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION,
        warmup_steps=5,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=50,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="steps",     # Estrategia de guardado
        save_steps=500,
        eval_strategy="steps",     # Estrategia de evaluación
        eval_steps=500,
        report_to="none",
        packing=False,
    ),
)
print("✅ Trainer configurado (Modern Stack)")

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/60000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.
✅ Trainer configurado (Modern Stack)


## Cell 7: 🔥 Entrenar

In [15]:
start_time = time.time()
print(f"🏋️ Entrenamiento iniciado: {datetime.now().strftime('%H:%M:%S')}")
print("=" * 50)

trainer_stats = trainer.train()

elapsed = time.time() - start_time
print("=" * 50)
print(f"✅ Entrenamiento completado en {elapsed/60:.1f} minutos")
print(f"📉 Training Loss Final: {trainer_stats.training_loss:.4f}")

🏋️ Entrenamiento iniciado: 15:36:23


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60,000 | Num Epochs = 1 | Total steps = 3,750
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss,Validation Loss
500,1.252988,1.230173
1000,1.203507,1.160746
1500,1.132081,1.115132
2000,1.106642,1.082388
2500,1.080255,1.058102
3000,1.077807,1.038220
3500,1.058463,1.026372


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


✅ Entrenamiento completado en 404.9 minutos
📉 Training Loss Final: 1.1487


## Cell 8: Evaluación Rápida

In [22]:
import random

FastLanguageModel.for_inference(model)

# Cargar test set real
with open(TEST_PATH, 'r') as f:
    test_data = json.load(f)

print(f"🧪 Test set: {len(test_data)} ejemplos")

# Evaluar sobre una muestra aleatoria
sample_size = 50
sample = random.sample(test_data, min(sample_size, len(test_data)))

correct = 0
total = len(sample)

print(f"📊 Evaluando {total} ejemplos del test set...")
print("=" * 70)

for item in sample:
    msgs = item["messages"]
    clause = [m["content"] for m in msgs if m["role"] == "user"][0]
    expected = [m["content"] for m in msgs if m["role"] == "assistant"][0]

    prompt = f"### Instruction:\n{clause}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=20, temperature=0.1)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = result.split("### Response:")[-1].strip().split("\n")[0]

    match = expected.lower() == response.lower()
    if match:
        correct += 1
    icon = "✅" if match else "❌"
    print(f"  {icon} Esperado: {expected:25s} | Predicción: {response}")

print("=" * 70)
print(f"📊 Accuracy: {correct}/{total} ({100*correct/total:.1f}%)")

🧪 Test set: 10000 ejemplos
📊 Evaluando 50 ejemplos del test set...
  ❌ Esperado: Entire Agreements         | Predicción: Misc
  ❌ Esperado: Waiver Of Jury Trials     | Predicción: Waivers.
  ❌ Esperado: Death                     | Predicción: Death Benefits
  ❌ Esperado: Change In Control         | Predicción: Change In Control.
  ✅ Esperado: Assignments               | Predicción: Assignments
  ✅ Esperado: Notices                   | Predicción: Notices
  ❌ Esperado: Enforceability            | Predicción: Misc
  ❌ Esperado: Disclosures               | Predicción: Records.
  ❌ Esperado: Binding Effects           | Predicción: Assignments
  ❌ Esperado: Effective Dates           | Predicción: Waivers
  ❌ Esperado: Headings                  | Predicción: 
  ❌ Esperado: Taxes                     | Predicción: 
  ❌ Esperado: Change In Control         | Predicción: Change
  ✅ Esperado: Taxes                     | Predicción: Taxes
  ❌ Esperado: Counterparts              | Predicción: Counte

## Cell 9: Exportar LoRA Adapters (~20MB)

In [23]:
LORA_OUTPUT = "/kaggle/working/lora_model"
model.save_pretrained(LORA_OUTPUT)
tokenizer.save_pretrained(LORA_OUTPUT)

# Calcular tamaño
lora_size = sum(os.path.getsize(os.path.join(LORA_OUTPUT, f)) for f in os.listdir(LORA_OUTPUT))
print(f"✅ LoRA adapters guardados en: {LORA_OUTPUT}")
print(f"💾 Tamaño: {lora_size / 1e6:.1f} MB")
print(f"📁 Archivos: {os.listdir(LORA_OUTPUT)}")

✅ LoRA adapters guardados en: /kaggle/working/lora_model
💾 Tamaño: 114.5 MB
📁 Archivos: ['tokenizer_config.json', 'tokenizer.json', 'README.md', 'adapter_model.safetensors', 'chat_template.jinja', 'adapter_config.json']


## Cell 10: Exportar GGUF (para uso local con llama.cpp/Ollama)

In [24]:
# Exportar a GGUF q4_k_m (buen balance calidad/tamaño)
GGUF_OUTPUT = "/kaggle/working/gguf_model"

try:
    model.save_pretrained_gguf(
        GGUF_OUTPUT,
        tokenizer,
        quantization_method="q4_k_m",
    )
    
    # Mostrar archivos generados
    for f in os.listdir(GGUF_OUTPUT):
        size = os.path.getsize(os.path.join(GGUF_OUTPUT, f)) / 1e9
        print(f"  {f}: {size:.2f} GB")
    print(f"✅ GGUF exportado a: {GGUF_OUTPUT}")
except Exception as e:
    print(f"⚠️ GGUF export falló (puede requerir más RAM): {e}")
    print("Los LoRA adapters siguen disponibles en /kaggle/working/lora_model")

Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Note: tokenizer.model not found (this is OK for non-SentencePiece models)



Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:42<00:00, 21.10s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/gguf_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...


[unsloth_zoo.llama_cpp|WARNING]Unsloth: Qwen2MoE num_experts patch target not found.


Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['/kaggle/working/gguf_model_gguf/Llama-3.2-3B-Instruct.F16.gguf']
Unsloth: [2] Converting GGUF f16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['/kaggle/working/gguf_model_gguf/Llama-3.2-3B-Instruct.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: llama.cpp/llama-cli --model /kaggle/working/gguf_model_gguf/Llama-3.2-3B-Instruct.Q4_K_M.gguf -p "why is the sky blue?"
Unsloth: Saved Ollama Modelfile to /kaggle/working/gguf_model_gguf/Modelfile
Unsloth: convert model to ollama format by running - ollama create model_name -f /kaggle/working/gguf_model_gguf/Modelfile
  .cache: 0.00 GB
  model-00001-of-00002.safetensors: 4.97 GB
  tokenizer_config.json: 0.00 GB
  model-00002-of-00002.safetensors: 1.46 GB
  tokenizer.json: 0.02 GB
  config.json: 0.00 GB
  mode

## Cell 11: Guardar Métricas & Resumen

In [26]:
# Guardar métricas del entrenamiento
metrics = {
    "model": MODEL_NAME,
    "dataset_path": DATASET_PATH,
    "dataset_size": len(raw_data),
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "training_loss": trainer_stats.training_loss,
    "training_time_minutes": elapsed / 60,
    "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
    "timestamp": datetime.now().isoformat(),
    "log_history": trainer_stats.metrics,
}

with open("/kaggle/working/training_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2, default=str)

print("📊 Resumen del Entrenamiento")
print("=" * 40)
print(f"  Modelo:          {MODEL_NAME}")
print(f"  Dataset:         {len(raw_data)} ejemplos")
print(f"  Epochs:          {EPOCHS}")
print(f"  Loss Final:      {trainer_stats.training_loss:.4f}")
print(f"  Tiempo:          {elapsed/60:.1f} min")
print(f"  GPU:             {torch.cuda.get_device_name(0)}")
print("=" * 40)
print("\n📁 Archivos de salida en /kaggle/working/:")
print("  - lora_model/          (LoRA adapters)")
print("  - gguf_model/          (GGUF para Ollama)")
print("  - training_metrics.json")
print("\n🎉 ¡Fine-tuning completado! Descarga los archivos desde la pestaña Output.")

📊 Resumen del Entrenamiento
  Modelo:          unsloth/Llama-3.2-3B-Instruct-bnb-4bit
  Dataset:         60000 ejemplos
  Epochs:          1
  Loss Final:      1.1487
  Tiempo:          404.9 min
  GPU:             Tesla T4

📁 Archivos de salida en /kaggle/working/:
  - lora_model/          (LoRA adapters)
  - gguf_model/          (GGUF para Ollama)
  - training_metrics.json

🎉 ¡Fine-tuning completado! Descarga los archivos desde la pestaña Output.
